# Load the dataset

In [1]:
run manifold_twins

In [2]:
%matplotlib ipympl

In [80]:
# Default settings for matplotlib figures
import matplotlib as mpl

# Choose how big to make figures. This will scale the text size.
mpl.rcParams['figure.figsize'] = (5, 4)

# Set the DPI. This will change how big things appear in Jupyter lab
mpl.rcParams['figure.dpi'] = 120

In [3]:
# a = ManifoldTwinsAnalysis(idr='HICKORY', verbosity=1)
a = ManifoldTwinsAnalysis(idr='CASCAD', verbosity=1)
# a = ManifoldTwinsAnalysis(idr='CASCAD', verbosity=1, bin_velocity=2000, max_count=100)

Loading dataset...
IDR:          CASCAD
Phase range: [-5.0, 5.0]
Center phase: 0.0
Bin velocity: 1000.0
Cutting SNe:  ['PTF12ecm']


/home/kboone/packages/conda/lib/python3.7/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)
100%|██████████| 438/438 [00:19<00:00, 22.49it/s]


# Interpolate the spectra

In [81]:
# Choose how much of the model to refit. For a full analysis, use 'refit' mode.
# If you already ran everything with the exact same configuration, 'cached' will
# load the previous result. If anything changed, 'cached_uncertainty' can be used
# to keep the uncertainty model from the last fit (which takes a long time to fit,
# and is very stable) but refit everything else. Don't use cached_uncertainty for
# the final analysis, it is only for debugging!

# fit_type = 'refit'              # Refit everything
# fit_type = 'cached'             # Use a full cached model that was previously fit.
fit_type = 'cached_uncertainty' # Use only the uncertainty from a cached model.

In [5]:
print("Modeling maximum spectra with fit type '%s'" % fit_type)

if fit_type == 'refit':
    a.model_maximum_spectra(use_cache=False)
elif fit_type == 'cached':
    a.model_maximum_spectra(use_cache=True)
elif fit_type == 'cached_uncertainty':
    a.model_maximum_spectra(use_cached_model_uncertainty=True)
else:
    print("Unknown fit type! Can't handle")
    
print("Done!")

Modeling maximum spectra with fit type 'cached_uncertainty'
Using saved interpolation result
Using cached model uncertainty, but refitting rest of model
Loaded cached stan model
Done!


# Read between the lines

In [6]:
a.read_between_the_lines()

print("Done!")

Loaded cached stan model
Masking 24/203 targets whose interpolation uncertainty power is more than 0.100 of the intrinsic power.
Done!


# Do embedding

In [7]:
a.do_embedding(n_components=3, n_neighbors=10)

# Apply standardization

## GP standardization

In [8]:
# a.apply_gp_standardization(hyperparameters=[0.1, 0.2, 0.5])
a.fit_gp()
a.plot_gp()
plt.xlabel('Component 0')
plt.ylabel('Component 1')

plt.colorbar(label='Magnitude residual')

Fitting GP hyperparameters...
Fit result:
      fun: -61.72812085784106
 hess_inv: array([[ 1.14037372e-04, -1.99752860e-05,  6.80852827e-03],
       [-1.99752860e-05,  6.82820803e-04,  1.51909292e-02],
       [ 6.80852827e-03,  1.51909292e-02,  1.80584487e+00]])
      jac: array([-7.15255737e-06,  4.29153442e-06,  0.00000000e+00])
  message: 'Optimization terminated successfully.'
     nfev: 140
      nit: 17
     njev: 28
   status: 0
  success: True
        x: array([0.08997931, 0.09584676, 2.3930576 ])
Fit NMAD:        0.08965478337441642
Fit std:         0.09960616591465518
(2500, 3)


FigureCanvasNbAgg()

In [9]:
a.apply_polynomial_standardization(1)

Fitted coefficients:
[-0.00961829 -0.09646324 -0.02681997 -0.03362577  0.00647974]
Fit NMAD:        0.09427576132984083
Fit std:         0.10001612733869826


In [11]:
a.calculate_salt_hubble_residuals()
a.apply_polynomial_standardization(1, kind='salt_raw')

Pass 0, MB=-19.149, alpha=0.129, beta=2.985
  -> new intrinsic_dispersion=0.119
Pass 1, MB=-19.149, alpha=0.129, beta=2.987
  -> new intrinsic_dispersion=0.119
Pass 2, MB=-19.149, alpha=0.129, beta=2.987
  -> new intrinsic_dispersion=0.119
Pass 3, MB=-19.149, alpha=0.129, beta=2.987
  -> new intrinsic_dispersion=0.119
Pass 4, MB=-19.149, alpha=0.129, beta=2.987
  -> new intrinsic_dispersion=0.119
SALT2 Hubble fit: 
    MB:    -19.14936035870967
    alpha: 0.12912169654736533
    beta:  2.986549046949107
    σ_int: 0.11893666801711794
    std:   0.1506544775204048
    NMAD:  0.10991458888441766
Fitted coefficients:
[-0.0976639   2.84279657 -0.04889293 -0.07488739  0.02255746]
Fit NMAD:        0.12614465898396188
Fit std:         0.12535210575388417


In [11]:
plt.figure()
plt.scatter(a.salt_phases[a.center_mask][a.good_mag_mask], a.corr_mags[a.good_mag_mask], label='Individual observations')
math.plot_binned_mean(a.salt_phases[a.center_mask][a.good_mag_mask], a.corr_mags[a.good_mag_mask], c='C2', label='Binned mean')
plt.xlabel('Phase of closest spectrum to maximum (days)')
plt.ylabel('Hubble residual (mag)')
plt.legend()

FigureCanvasNbAgg()

In [12]:
params = [0.1, 0.2, 5]

a.calculate_salt_hubble_residuals()

print("std: ", a._calculate_gp_dispersion(params, np.std))
print("NMAD:", a._calculate_gp_dispersion(params, math.nmad))
print("std: ", a._calculate_gp_dispersion(params, np.std, kind='salt'))
print("NMAD:", a._calculate_gp_dispersion(params, math.nmad, kind='salt'))

Pass 0, MB=-19.149, alpha=0.129, beta=2.985
  -> new intrinsic_dispersion=0.119
Pass 1, MB=-19.149, alpha=0.129, beta=2.987
  -> new intrinsic_dispersion=0.119
Pass 2, MB=-19.149, alpha=0.129, beta=2.987
  -> new intrinsic_dispersion=0.119
Pass 3, MB=-19.149, alpha=0.129, beta=2.987
  -> new intrinsic_dispersion=0.119
Pass 4, MB=-19.149, alpha=0.129, beta=2.987
  -> new intrinsic_dispersion=0.119
SALT2 Hubble fit: 
    MB:    -19.14936035870967
    alpha: 0.12912169654736533
    beta:  2.986549046949107
    σ_int: 0.11893666801711794
    std:   0.1506544775204048
    NMAD:  0.10991458888441766
std:  0.1002223245408711
NMAD: 0.09136292867040062
std:  0.12466507915583643
NMAD: 0.09291191249364794


In [13]:
plt.figure()
# plt.scatter(a.trans[:, 0], a.trans[:, 1], c=a.salt_x1)
# plt.scatter(a.trans[a.good_mag_mask, 0] + a.trans[a.good_mag_mask, 1], a.salt_hr)
mask = a.salt_mask & a.redshift_color_mask & a.interp_mask
plt.scatter(a.trans[mask, 0], a.salt_hr[mask])
math.plot_binned_mean(a.trans[mask, 0], a.salt_hr[mask], c='C2', lw=2)

plt.figure()
plt.hist(a.salt_hr[mask & (a.trans[:, 0] < 2)], 10, (-0.6, 0.4), alpha=0.3)
plt.hist(a.salt_hr[mask & (a.trans[:, 0] > 2)], 10, (-0.6, 0.4), alpha=0.3)

FigureCanvasNbAgg()

FigureCanvasNbAgg()

(array([1., 0., 4., 2., 6., 4., 0., 1., 0., 0.]),
 array([-6.00000000e-01, -5.00000000e-01, -4.00000000e-01, -3.00000000e-01,
        -2.00000000e-01, -1.00000000e-01,  1.11022302e-16,  1.00000000e-01,
         2.00000000e-01,  3.00000000e-01,  4.00000000e-01]),
 <a list of 10 Patch objects>)

In [18]:
plt.figure()
plt.scatter(a.trans[a.mag_cut & a.train_cut, 1], a.salt_x1[a.mag_cut & a.train_cut])
# np.std(a.salt_hr[a.mag_cut & a.train_cut])

# plt.scatter(a.trans[a.mag_cut, 0], a.salt_hr[a.mag_cut])

FigureCanvasNbAgg()

AttributeError: 'ManifoldTwinsAnalysis' object has no attribute 'mag_cut'

In [36]:
a.do_blondin_plot()
# plt.xlabel('First Isomap Component')
# plt.xlabel('Second Isomap Component')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [315]:
plt.savefig('./branch.eps')

## Polynomial standardization

In [38]:
# This doesn't work in >3 dimensions.
a.apply_polynomial_standardization(degree=1)

Fitted color slope: -0.06609644511440725
Fitted coefficients:
[[[-0.01248886 -0.00134519]
  [-0.03924846  0.00594632]]

 [[-0.02548631 -0.01135962]
  [-0.01682301  0.01206172]]]
Fit NMAD:        0.0906907952718601
Fit std:         0.09581296097942876


## Comparison to original twins

In [15]:
a.plot_twin_pairings()
plt.ylim(0, None)

RMS  20%: 0.11176643991362427
NMAD 20%: 0.061700413481005106


FigureCanvasNbAgg()

(0, 0.3240731639672975)

In [192]:
plt.savefig('./twin_dispersion.eps')

## Embedding tests

In [16]:
a.do_embedding(n_components=3, n_neighbors=10)
leakage_matrix = a.plot_twin_distances()

FigureCanvasNbAgg()

,To Best 10% of twins,To 10-20%,To 20-50%,To Worst 50% of twins
From Best 10% of twins,0.706905,0.241429,0.051666,0.000000
From 10-20%,0.195652,0.403382,0.399517,0.001449
From 20-50%,0.030425,0.111880,0.678042,0.179652
From Worst 50% of twins,0.001256,0.003864,0.102975,0.891905


In [241]:
vals = [
    0.052800 + 0.052028,
    0.015983 + 0.020392,
    0.001874 + 0.006173,
    0.000772 + 0.003527,
    0.000220 + 0.001874,
]
plt.figure()
plt.scatter(np.arange(len(vals)) + 1, vals)
plt.ylim(0, 0.1)
plt.xlabel('Number of components (in addition to color)')
plt.ylabel('Fraction of non-twins confused as twins')

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

Text(0, 0.5, 'Fraction of non-twins confused as twins')

In [311]:
plt.savefig('twins_confusion.eps')

In [8]:
plt.xlabel('Recovered twins percentile from the embedded space')
plt.ylabel('Fraction in bin')

Text(42.722222222222214, 0.5, 'Fraction in bin')

In [411]:
x = np.arange(a.trans.shape[1])
plt.figure()
var = np.var(a.trans, axis=0)
plt.scatter(x, var, label='Component variances')
plt.axhline(0.47, label='Expected noise level')
plt.xlabel('Component number')
plt.ylabel('Explained variance in "twinness" $\\xi$')
plt.legend()

FigureCanvasNbAgg()

In [400]:
plt.savefig('./component_variances.eps')

In [63]:
a.do_embedding(n_components=10, n_neighbors=10)

In [95]:
vals_same = []
vals_other = []
all_dists = np.zeros((len(a.spectra), len(a.spectra)))
trans_dists = np.zeros((len(a.spectra), len(a.spectra)))

same_mask = np.zeros((len(a.spectra), len(a.spectra)), dtype=bool)

frac_spectra = a.scale_flux / a.mean_spectrum

for ii, i in enumerate(a.spectra):
    for jj, j in enumerate(a.spectra):
        if ii <= jj:
            continue

        fi = frac_spectra[ii]
        fj = frac_spectra[jj]
        
        xi = np.sqrt(np.sum((fi - fj)**2))

        if i.target == j.target:
            vals_same.append(xi)
            same_mask[ii, jj] = True
            same_mask[jj, ii] = True
        else:
            vals_other.append(xi)
            
        all_dists[ii, jj] = xi
        all_dists[jj, ii] = xi
        
        trans_xi = np.sqrt(np.sum((a.trans[ii] - a.trans[jj])**2))
        trans_dists[ii, jj] = trans_xi
        trans_dists[jj, ii] = trans_xi

In [128]:
flat_dists = []
flat_iso_dists = []
flat_trans_dists = []

for ii, i in enumerate(a.spectra):
    for jj, j in enumerate(a.spectra):
        if ii <= jj:
            continue
        flat_dists.append(all_dists[ii, jj])
        flat_iso_dists.append(a.iso.dist_matrix_[ii, jj])
        flat_trans_dists.append(trans_dists[ii, jj])
        
flat_dists = np.array(flat_dists)
flat_iso_dists = np.array(flat_iso_dists)
flat_trans_dists = np.array(flat_trans_dists)

NameError: name 'all_dists' is not defined

In [97]:
twins_percentile = 20
twins_mask = flat_iso_dists < np.percentile(flat_iso_dists, 20)
print(np.std((flat_trans_dists - flat_iso_dists)[twins_mask]))
print(np.std(flat_iso_dists[twins_mask]))

0.3262092073428555
0.39775095824002704


## Standardization tests

In [55]:
a.targets[a.train_cut][a.mag_cut[a.train_cut] & (a.corr_mags < -0.5)]

array([Target(name="PTF12ecm")], dtype=object)

In [52]:
plt.figure()
plt.scatter(a.trans[a.train_cut & a.mag_cut, 1], a.corr_mags[a.mag_cut[a.train_cut]], c=a.trans[a.train_cut & a.mag_cut, 0])

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [361]:
plt.figure()
plt.scatter(a.mags[a.mag_cut[a.train_cut]], (a.mags - a.corr_mags)[a.mag_cut[a.train_cut]])
plt.plot([-0.4, 0.3], [-0.4, 0.3])

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [362]:
a.plot_gp(axis_1=0, axis_2=1)
plt.colorbar(label='Magnitude residual')
plt.xlabel('Component 0')
plt.ylabel('Component 1')

(2500, 3)


/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

Text(0, 0.5, 'Component 1')

In [363]:
a.scatter(a.mags, a.train_cut, a.mag_cut, vmin=-0.2, vmax=0.2, axis_1=0, axis_2=1, label='Magnitude residual', cmap=plt.cm.coolwarm)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [364]:
a.scatter(a.corr_mags, a.train_cut, a.mag_cut, vmin=-0.2, vmax=0.2, axis_1=0, axis_2=1, label='Magnitude residual', cmap=plt.cm.coolwarm)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [365]:
cc = a.redshifts > 0.1
print(a.targets[cc])
print(a.mags[cc[a.train_cut]])
print(a.corr_mags[cc[a.train_cut]])

[Target(name="SNF20060618-023") Target(name="SNF20070528-003")
 Target(name="SNF20070912-000")]
[-0.28580157 -0.50868427 -0.32860607]
[-0.05155959 -0.22002627 -0.05287224]


In [152]:
a.apply_gp_standardization(hyperparameters=[0.09772533, 0.19172349, 2., 0.1, 3.], phase=True)

Using fixed GP hyperparameters...
Fit NMAD:        0.09618198468026534
Fit std:         0.13151859789255274


In [153]:
a.apply_gp_standardization(hyperparameters=[0.1, 0.3, 2.])

Using fixed GP hyperparameters...
Fit NMAD:        0.10905397363474512
Fit std:         0.13388293303568655


In [154]:
plt.figure()
first_phase = np.array([len(i.target.spectra) for i in a.spectra])
plt.scatter(a.salt_phases[a.mag_cut & a.train_cut], a.corr_mags[a.mag_cut[a.train_cut]], c=first_phase[a.mag_cut & a.train_cut], vmin=5, vmax=15)
plt.colorbar()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 255 but corresponding boolean dimension is 151

In [161]:
s2n_uu = np.array([i.get_signal_to_noise(3300, 3800) for i in a.raw_spectra])
s2n_ii = np.array([i.get_signal_to_noise(8100, 8600) for i in a.raw_spectra])
plt.figure()
plt.scatter(s2n_uu[a.mag_cut & a.train_cut], a.corr_mags[a.mag_cut[a.train_cut]])

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 255 but corresponding boolean dimension is 151

In [277]:
len(a.train_cut)

191

In [55]:
print(math.nmad(a.salt_hr[a.mag_cut & a.train_cut & a.salt_cut]))
print(np.std(a.corr_mags[(a.mag_cut & a.salt_cut)[a.train_cut]]))

AttributeError: 'ManifoldTwinsAnalysis' object has no attribute 'salt_hr'

In [34]:
plt.figure()

plt.scatter(a.redshifts[a.train_cut], a.corr_mags, c=a.colors[a.train_cut], vmin=0, vmax=0.5)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [163]:
a.scatter(a.corr_mags, a.train_cut, a.mag_cut, vmin=-0.2, vmax=0.2, axis_1=0, axis_2=1, axis_3=2, label='Magnitude residual', cmap=plt.cm.coolwarm)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [164]:
a.scatter(a.mags, a.train_cut, a.mag_cut, vmin=-0.2, vmax=0.2, axis_1=0, axis_2=1, axis_3=2, label='Magnitude residual', cmap=plt.cm.coolwarm)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [192]:
a.scatter(a.mags, a.train_cut, a.mag_cut, vmin=-0.2, vmax=0.2, axis_1=0, axis_2=1, axis_3=None, label='Magnitude residual', cmap=plt.cm.coolwarm)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [166]:
a.scatter(a.corr_mags, a.train_cut, a.mag_cut, vmin=-0.2, vmax=0.2, axis_1=0, axis_2=1, axis_3=None, label='Magnitude residual', cmap=plt.cm.coolwarm)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [191]:
a.scatter(a.salt_x1, a.train_cut, None, vmin=-1., vmax=1., axis_1=0, axis_2=1, axis_3=None, label='SALT X1', cmap=plt.cm.coolwarm)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [169]:
plt.figure()
plt.scatter(a.trans[a.train_cut & a.mag_cut, 0], a.corr_mags[a.mag_cut[a.train_cut]], c=a.trans[a.train_cut & a.mag_cut, 1])

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [84]:
plt.figure()
plt.scatter(a.trans[:, 0], a.redshifts)

FigureCanvasNbAgg()

In [85]:
plt.figure()
a.raw_spectra[a.redshifts > 0.1][0].plot()

FigureCanvasNbAgg()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [143]:
a.spectra[a.train_cut][a.mag_cut[a.train_cut] & (np.abs(a.corr_mags) > 0.3)]

array([], dtype=object)

In [121]:
math.nmad(a.corr_mags[(a.mag_cut)[a.train_cut]])

0.08760371254393381

In [126]:
plt.figure()
s2n_uu = np.array([i.get_signal_to_noise(3300, 3800) for i in a.spectra])
s2n_ii = np.array([i.get_signal_to_noise(8100, 8600) for i in a.spectra])
s2n2 = np.array([i.get_snf_signal_to_noise('u') for i in a.spectra])
# plt.scatter(s2n[a.mag_cut & a.train_cut], s2n2[a.mag_cut & a.train_cut], c=a.corr_mags[a.mag_cut[a.train_cut]], cmap=plt.cm.coolwarm, vmin=-0.2, vmax=0.2)
plt.scatter(s2n_uu[a.train_cut & a.mag_cut], a.corr_mags[a.mag_cut[a.train_cut]], c=s2n_ii[a.train_cut & a.mag_cut])

FigureCanvasNbAgg()

In [263]:
plt.figure()
plt.scatter(a.colors[a.train_cut & a.mag_cut], a.corr_mags[a.mag_cut[a.train_cut]])
plt.xlabel('Measured color')
plt.ylabel('Manifold twins Hubble Residual')

FigureCanvasNbAgg()

Text(0, 0.5, 'Manifold twins Hubble Residual')

In [93]:
plt.figure()
plt.scatter(a.trans[a.mag_cut & a.train_cut, 0], a.corr_mags[a.mag_cut[a.train_cut]])

FigureCanvasNbAgg()

## Look at how spectra from the same target get embedded (must run analysis for all spectra)

In [309]:
a.plot_same_target_pairings()

FigureCanvasNbAgg()

# Host galaxy correlations

In [18]:
a.load_host_data()
a.calculate_salt_hubble_residuals()

# a.fit_gp(kind='salt')
a.apply_polynomial_standardization(kind='twins')

Pass 0, MB=-19.149, alpha=0.129, beta=2.985
  -> new intrinsic_dispersion=0.119
Pass 1, MB=-19.149, alpha=0.129, beta=2.987
  -> new intrinsic_dispersion=0.119
Pass 2, MB=-19.149, alpha=0.129, beta=2.987
  -> new intrinsic_dispersion=0.119
Pass 3, MB=-19.149, alpha=0.129, beta=2.987
  -> new intrinsic_dispersion=0.119
Pass 4, MB=-19.149, alpha=0.129, beta=2.987
  -> new intrinsic_dispersion=0.119
SALT2 Hubble fit: 
    MB:    -19.14936035870967
    alpha: 0.12912169654736533
    beta:  2.986549046949107
    σ_int: 0.11893666801711794
    std:   0.1506544775204048
    NMAD:  0.10991458888441766
Fitted coefficients:
[-0.00961829 -0.09646324 -0.02681997 -0.03362577  0.00647974]
Fit NMAD:        0.09427576132984083
Fit std:         0.10001612733869826


In [19]:
a.plot_host(threshold=-10.8)
# a.plot_host(cut=a.mag_cut & (a.colors > -0.1))
# a.plot_host()

interactive(children=(Dropdown(description='variable', options=('lmass', 'lmass_up', 'lmass_low', 'lsfr', 'lsf…

In [23]:
plt.figure()
plt.scatter(a.salt_x1[a.good_salt_mask], a.corr_mags[a.good_salt_mask])

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [25]:
plt.figure()
plt.scatter(a.host_data['lssfr'][a.salt_mask], a.salt_hr[a.salt_mask])

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [97]:
host_results = {
    # 'Local Mass':  [0.008,  0.04,   -0.02,  0.028],
    # 'Local SFR':   [0.013,  -0.007, -0.018, 0.015],
    # 'Local SSFR':  [0.058,  0.0956, 0.029,  0.070],
    # 'Global Mass': [0.0258,  0.061, -0.0136,-0.008],
    'Local Mass':  [-0.0041,  0.0488],
    'Local SSFR':  [0.0164,  0.0744],
    'Global Mass': [-0.0121,  0.0517],
}

err = 0.025

plt.figure()
labels = []
for idx, (key, values) in enumerate(host_results.items()):
    for val_idx, value in enumerate(values):
        if val_idx >= 2:
            continue
        marker = 'oovv'[val_idx]
        # color = 'C%d' % (val_idx % 2)
        color = 'C%d' % (val_idx % 2)
        if idx == 0:
            label = ['Manifold twins', 'SALT2', 'Manifold twins color cut', 'SALT2 color cut'][val_idx]
        else:
            label = None
        gap = 0.1
        # xpos = idx - 3.*gap + 2*gap * val_idx + gap * (val_idx // 2)
        # xpos = idx - 1.5*gap + gap * val_idx
        xpos = idx - 0.5*gap + gap * val_idx
        plt.errorbar(xpos, value, err, c=color, alpha=1.)
        plt.plot(xpos, value, marker=marker, c=color, label=label)
        
    labels.append(key)

plt.xticks(np.arange(len(labels)), labels=labels)
plt.axhline(0., c='k')
plt.xlim(-0.5, 2.5)
# plt.ylim(-0.05, 0.13)

plt.ylabel('Step size (mag)')
plt.legend(loc=2)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [231]:
plt.savefig('host_correlations.eps')

In [223]:
host_results = {
    # 'Local Mass':  [0.008,  0.04,   -0.02,  0.028],
    # 'Local SFR':   [0.013,  -0.007, -0.018, 0.015],
    # 'Local SSFR':  [0.058,  0.0956, 0.029,  0.070],
    # 'Global Mass': [0.0258,  0.061, -0.0136,-0.008],
    'Local Mass':  [0.008,  0.04,   -0.004, 0.051],
    'Local SFR':   [0.013,  -0.007, 0.0047, -0.031],
    'Local SSFR':  [0.058,  0.0956, 0.027, 0.065],
    'Global Mass': [0.0258,  0.061, -0.001, 0.040],
}

err = 0.025

plt.figure()
labels = []
for idx, (key, values) in enumerate(host_results.items()):
    for val_idx, value in enumerate(values):
        if val_idx >= 2:
            continue
        marker = 'oovv'[val_idx]
        # color = 'C%d' % (val_idx % 2)
        color = 'C%d' % (val_idx % 2)
        if idx == 0:
            label = ['Manifold twins', 'SALT2', 'Manifold twins color cut', 'SALT2 color cut'][val_idx]
        else:
            label = None
        gap = 0.1
        # xpos = idx - 3.*gap + 2*gap * val_idx + gap * (val_idx // 2)
        # xpos = idx - 1.5*gap + gap * val_idx
        xpos = idx - 0.5*gap + gap * val_idx
        plt.errorbar(xpos, value, err, c=color, alpha=0.3)
        plt.plot(xpos, value, marker=marker, c=color, label=label)
        
    labels.append(key)

plt.xticks(np.arange(len(labels)), labels=labels)
plt.axhline(0., c='k')
# plt.ylim(-0.05, 0.13)

plt.ylabel('Step size (mag)')
plt.legend(loc=2)

FigureCanvasNbAgg()

In [28]:
plt.figure()

c1 = (a.host_data['lssfr'][a.train_cut[a.host_mask]] > -11)[:, 0]
c2 = (a.corr_mags[a.host_mask[a.train_cut]] > 0.05)
cc = c1 + 2*c2

x = a.trans[:, 0][a.train_cut & a.host_mask]
y = a.trans[:, 1][a.train_cut & a.host_mask]
plt.scatter(x, y, c=cc, cmap=plt.cm.jet)
plt.colorbar()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [32]:
a.host_mask & a.train_cut

array([False,  True, False,  True, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True, False,
        True, False,  True, False, False,  True, False, False, False,
        True, False, False, False, False, False, False,  True,  True,
        True,  True, False,  True,  True, False, False, False, False,
        True,  True, False, False,  True, False, False,  True, False,
       False, False, False,  True, False, False, False, False, False,
       False,  True,  True, False, False,  True, False, False,  True,
        True, False, False, False, False, False, False,  True, False,
        True, False,  True, False, False,  True, False, False, False,
       False, False,  True,  True,  True, False,  True,  True, False,
        True,  True,  True, False,  True, False, False,  True,  True,
       False, False,  True,  True, False, False,  True,  True, False,
       False, False,

In [140]:
plt.figure()

lssfr = a.host_data['lssfr'][a.train_cut[a.host_mask]]
mag1 = a.salt_hr[a.host_mask & a.train_cut]
mag2 = a.corr_mags[a.host_mask[a.train_cut]]

for i in range(len(lssfr)):
    bigger = mag1[i] > mag2[i]
    if bigger:
        c = 'C3'
    else:
        c = 'C0'
    
    plt.arrow(lssfr[i], mag1[i], 0, mag2[i] - mag1[i], width=0.01, head_width=0.1, head_length=0.03, edgecolor=c, facecolor=c)
    
plt.xlim(np.min(lssfr) - 0.5, np.max(lssfr) + 0.5)
plt.ylim(-0.4, 0.4)

plt.figure()
plt.scatter(lssfr, mag1)

plt.figure()
plt.scatter(lssfr, mag2)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

# Plot a slice through the twins manifold

In [218]:
scan_component = 2
dist_limit = 0.
loc = np.array([0.0, 0])

other_trans = np.delete(a.trans, scan_component, axis=1)
dists = np.sqrt(np.sum((other_trans - loc)**2, axis=1))

scan_cut = dists < dist_limit

min_comp = np.min(a.trans[scan_cut, scan_component])
max_comp = np.max(a.trans[scan_cut, scan_component])
cmap = plt.cm.viridis
# cmap = plt.cm.coolwarm

plt.figure()
for spec, val in zip(a.scale_flux[scan_cut], a.trans[scan_cut, scan_component]):
    plt.plot(a.wave, spec, c=cmap((val - min_comp) / (max_comp - min_comp)))
    
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Normalized flux')

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_comp, vmax=max_comp))
sm._A = []
plt.colorbar(sm, label='Value of first component')
# plt.xlim(5000, 6500)
# plt.ylim(0, 15)
# plt.xlim(3300, 4300)

ValueError: zero-size array to reduction operation minimum which has no identity

In [278]:
plt.savefig('./slice.eps')

# Test SALT2 Hubble residuals

In [19]:
a.calculate_salt_hubble_residuals()

SALT2 Hubble fit: 
    alpha: 0.13456082620595397
    beta:  3.101659006808755
    std:   0.14863885217406037
    NMAD:  0.11079904413117943


In [244]:
plt.figure()
salt_colors = np.array([i.target['salt2.Color'] for i in a.spectra])
salt_x1 = np.array([i.target['salt2.X1'] for i in a.spectra])
# plt.scatter(salt_x1, a.colors - 3.1*salt_colors)
# plt.scatter((a.colors - 3.1*salt_colors)[a.train_cut & a.mag_cut], a.corr_mags[a.mag_cut[a.train_cut]])

x = a.trans[:, 0]

# plt.scatter(x[a.train_cut & a.mag_cut & a.salt_cut], a.corr_mags[(a.salt_cut & a.mag_cut)[a.train_cut]])
# plt.scatter(x[a.train_cut & a.mag_cut & a.salt_cut], a.salt_hr[(a.salt_cut & a.mag_cut & a.train_cut)])
plt.scatter(x[a.mag_cut & a.salt_cut], a.salt_hr[(a.salt_cut & a.mag_cut)])
# plt.scatter(salt_colors[a.train_cut & a.mag_cut], a.corr_mags[a.mag_cut[a.train_cut]])

plt.xlabel('Component 0')
plt.ylabel('SALT 2 Hubble residual')

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

Text(0, 0.5, 'SALT 2 Hubble residual')

In [151]:
plt.figure()
plt.scatter(a.trans[:, 1], a.salt_x1, c=a.trans[:, 0])

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [76]:
a.targets[a.train_cut][(a.corr_mags > 0.4) & (a.mag_cut)[a.train_cut]]

array([], dtype=object)

In [77]:
plt.figure()
plt.scatter(a.corr_mags[(a.mag_cut & a.salt_cut)[a.train_cut]], a.salt_hr[a.mag_cut & a.train_cut & a.salt_cut])

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [78]:
np.std(a.salt_hr[a.mag_cut & a.salt_cut & a.train_cut & (a.trans[:, 0] < 2)])

0.1390800654979601

In [80]:
plt.figure()
plt.scatter(a.trans[:, 0][a.mag_cut & a.salt_cut], a.salt_hr[a.mag_cut & a.salt_cut], s=10, label='Individual measurements')
math.plot_binned_mean(a.trans[:, 0][a.mag_cut & a.salt_cut], a.salt_hr[a.mag_cut & a.salt_cut], 10, equal_bin_counts=True, mode='error', label='Binned mean')
# plt.scatter(a.trans[:, 0][a.mag_cut & a.salt_cut & a.train_cut], a.salt_hr[a.mag_cut & a.salt_cut & a.train_cut])
plt.xlabel('First Isomap component')
plt.ylabel('Hubble Residuals corrected for SALT2 $x_1$ and color')
plt.legend()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [308]:
plt.savefig('./salt_hr_vs_component_0.eps')

In [470]:
math.nmad(a.corr_mags[(a.salt_cut & a.mag_cut)[a.train_cut]])

0.10040325524385625

# Modeling the full timeseries

In [30]:
all_phases = []
all_trans = []
all_fluxes = []
all_spectra = []

for idx in tqdm.tqdm(range(len(a.targets))):
    target = a.targets[idx]
    # scale = a.applied_scale[idx]
    # scale = a.scale_flux[idx] / a.interpolation_result['target_flux'][idx]
    scale = 1 / a.model_scales[idx]
    trans = a.trans[idx]

    for raw_spectrum in target.spectra:
        spectrum = raw_spectrum.bin_by_velocity(1000).apply_scale(scale)
        
        all_phases.append(spectrum.phase)
        all_fluxes.append(spectrum.flux)
        all_trans.append(trans)
        all_spectra.append(spectrum)
        
all_phases = np.array(all_phases)
all_fluxes = np.array(all_fluxes)
all_trans = np.array(all_trans)
all_spectra = np.array(all_spectra)

100%|██████████| 191/191 [02:17<00:00,  2.30it/s]


In [31]:
plt.figure()
plt.scatter(all_phases, all_fluxes[:, 20], c=all_trans[:, 0])

FigureCanvasNbAgg()

In [32]:
wave_idx = 50

a.scatter(a.scale_flux[:, wave_idx], label='Flux')
plt.title('%d $\AA$' % a.wave[wave_idx])

FigureCanvasNbAgg()

Text(0.5, 1.0, '3903 $\\AA$')

In [33]:
interp_x = np.hstack([all_phases[:, None], all_trans])

In [34]:
# Take ratios to the median spectrum and see how components affect things.
def do_plot(trans_idx, wave_idx):
    order = np.argsort(all_phases)
    x = all_trans[order, trans_idx]
    y = all_fluxes[order, wave_idx]

    mean_func = math.windowed_median(y)
    scale_y = y / mean_func

    plt.figure()
    plt.scatter(x, scale_y, s=10, alpha=0.2)
    math.plot_binned_median(x, scale_y, mode='error')
    plt.ylim(0, 2)
    # plt.ylim(-1, 2)
    # plt.xlim(-30, 50)
    
from ipywidgets import interact
interact(do_plot, trans_idx=(0, all_trans.shape[1]-1), wave_idx=(0, all_fluxes.shape[1]-1))

interactive(children=(IntSlider(value=1, description='trans_idx', max=2), IntSlider(value=143, description='wa…

<function __main__.do_plot(trans_idx, wave_idx)>

## Spline model

In [35]:
from scipy.interpolate import BSpline

min_phase = -15
max_phase = 60

t = np.arange(min_phase, max_phase, 5)
t = np.hstack([t[0], t[0], t, t[-1], t[-1]])
k = 3
len_c = len(t) - k - 1

def super_spline(all_c, phases, trans):
    use_c = all_c

    center_spl = BSpline(t, use_c[0], k)
    x0_spl = BSpline(t, use_c[1], k)
    x1_spl = BSpline(t, use_c[2], k)
    x2_spl = BSpline(t, use_c[3], k)
    x00_spl = BSpline(t, use_c[4], k)
    x01_spl = BSpline(t, use_c[5], k)
    x02_spl = BSpline(t, use_c[6], k)
    x11_spl = BSpline(t, use_c[7], k)
    x12_spl = BSpline(t, use_c[8], k)
    x22_spl = BSpline(t, use_c[9], k)
    
    c0 = trans[:, 0] / 10.
    c1 = trans[:, 1] / 10.
    c2 = trans[:, 2] / 10.
    
    model = (
        center_spl(phases) +
        c0 * x0_spl(phases) +
        c1 * x1_spl(phases) +
        c2 * x2_spl(phases) +
        c0 * c0 * x00_spl(phases) +
        c0 * c1 * x01_spl(phases) +
        c0 * c2 * x02_spl(phases) +
        c1 * c1 * x11_spl(phases) +
        c1 * c2 * x12_spl(phases) +
        c2 * c2 * x22_spl(phases)
    )
    
    # print(np.max(mag_model), np.min(mag_model))
    # model = 10**(-0.4*mag_model)
    
    return model

In [36]:
def build_spline_basis(phases, trans):
    basis = []
    
    for i in range(10):
        for j in range(len_c):
            impulse_coeffs = np.zeros((10, len_c))
            impulse_coeffs[i, j] = 1.
            spl = super_spline(impulse_coeffs, use_phases, use_trans)
            basis.append(spl)
            
    basis = np.array(basis)
    
    return basis

In [37]:
cut = (all_phases > min_phase) & (all_phases < max_phase)
use_phases = all_phases[cut]
use_trans = all_trans[cut]
basis = build_spline_basis(use_phases, use_trans)
print(np.max(basis), np.min(basis))

6.715901288398874 -9.521467539915262


In [38]:
all_spl_c = []
for wave_idx in range(len(a.wave)):
    use_flux = all_fluxes[cut, wave_idx]
    coef, residuals, rank, s = np.linalg.lstsq(basis.T, use_flux, rcond=None)
    spl_c = coef.reshape((10, len_c))
    all_spl_c.append(spl_c)
all_spl_c = np.array(all_spl_c)

In [39]:
np.max(a.trans)

8.41951815955247

In [40]:
vary_idx = 1
center_loc = [0, 0, 0.]

center_loc = np.array(center_loc, dtype=float)
deltas = all_trans - center_loc
diff = np.delete(deltas, vary_idx, axis=1)
dist = np.sqrt(np.sum(diff**2, axis=1))
close_cut = dist < 1.

close_trans = all_trans[close_cut, vary_idx]

min_vary = np.min(close_trans)
max_vary = np.max(close_trans)
diff = max_vary - min_vary
min_vary -= 0.1*diff
max_vary += 0.1*diff


plt.figure()
for plot_loc in np.linspace(min_vary, max_vary, 10):
    plot_t = np.linspace(-15, 60, 1000)
    loc = center_loc.copy()
    loc[vary_idx] = plot_loc
    pred_trans = np.array([loc] * len(plot_t))
    
    model = super_spline(spl_c, plot_t, pred_trans)
    
    plt.plot(
        plot_t, model,
        c=plt.cm.coolwarm((plot_loc - min_vary) / (max_vary - min_vary)),
    )

plt.scatter(all_phases[close_cut], all_fluxes[close_cut, wave_idx], c=all_trans[close_cut, vary_idx],
            vmin=min_vary, vmax=max_vary, cmap=plt.cm.coolwarm)
plt.xlim(-20, 40)
# plt.ylim(-5, 25)

FigureCanvasNbAgg()

(-20, 40)

In [41]:
# Residuals
# phases = np.array([i.phase for i in a.spectra])
phases = np.zeros(len(a.targets))
a.scatter(a.scale_flux[:, wave_idx], cmap=plt.cm.coolwarm)
residuals = a.scale_flux[:, wave_idx] - super_spline(spl_c, phases, a.trans)
scale = np.max(np.abs(residuals))
a.scatter(residuals, cmap=plt.cm.coolwarm, vmin=-scale, vmax=scale)

plt.figure()
plt.scatter(a.trans[:, 2], residuals)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [42]:
plt.figure()
plt.scatter(residuals[a.train_cut & a.mag_cut], a.corr_mags[a.mag_cut[a.train_cut]])

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [49]:
plt.figure()
plt.scatter(all_trans[:, 1], all_trans[:, 2])

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [53]:
# Reproduced spectra
for vary_idx in range(3):
    # center_loc = [-1.4, -0.7, 0.]
    # center_loc = [-0.5, 0.5, 0]
    center_loc = [0, 0.5, 0]
    phase = 0

    center_loc = np.array(center_loc, dtype=float)
    deltas = all_trans - center_loc
    diff = np.delete(deltas, vary_idx, axis=1)
    dist = np.sqrt(np.sum(diff**2, axis=1))
    close_cut = dist < 1.

    close_trans = all_trans[close_cut, vary_idx]

    # min_vary = np.clip(np.min(close_trans), -5, -2)
    # max_vary = np.clip(np.max(close_trans), 2, 5)
    min_vary = np.min(close_trans)
    max_vary = np.max(close_trans)
    diff = max_vary - min_vary
    min_vary += 0.1*diff
    max_vary -= 0.1*diff
    vary_loc = np.linspace(min_vary, max_vary, 10)

    print(min_vary, max_vary)

    plt.figure(figsize=(8, 6))
    models = []
    for wave_idx in range(len(a.wave)):
        plot_t = np.ones(len(vary_loc)) * phase
        plot_loc = np.tile(center_loc, (10, 1))
        plot_loc[:, vary_idx] = vary_loc

        model = super_spline(all_spl_c[wave_idx], plot_t, plot_loc)

        models.append(model)

    models = np.array(models)
    for model, model_loc in zip(models.T, vary_loc):
        plt.plot(a.wave, model, c=plt.cm.coolwarm((model_loc - min_vary) / (max_vary - min_vary)))

    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Flux')
    plt.title('Component %d' % vary_idx)


        # plt.plot(
            # plot_t, model,
            # c=plt.cm.coolwarm((plot_loc - min_vary) / (max_vary - min_vary)),
        # )

    # plt.scatter(all_phases[close_cut], all_fluxes[close_cut, wave_idx], c=all_trans[close_cut, vary_idx],
                # vmin=min_vary, vmax=max_vary, cmap=plt.cm.coolwarm)
    # plt.xlim(-20, 40)
    # plt.ylim(-5, 25)

-3.446814751127695 3.675216663579458


/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

-3.356471377253979 1.793067745943505


/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

-1.0119235169273137 3.1224048892665506


/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [224]:
from idrtools.tools import snf_filters
from idrtools.spectrum import _get_snf_magnitude

lc_phases = np.linspace(min_phase, max_phase, 100)

def calc_lc(band, loc):
    models = []
    min_wave, max_wave = snf_filters[band]

    min_wave_idx = np.min(np.where(a.wave > min_wave))
    max_wave_idx = np.max(np.where(a.wave < max_wave))
    for wave_idx in range(min_wave_idx, max_wave_idx+1):
        plot_loc = np.tile(loc, (len(lc_phases), 1))
        model = super_spline(all_spl_c[wave_idx], lc_phases, plot_loc)
        models.append(model)

    models = np.array(models)
    bandmag = []
    for model in models.T:
        bandmag.append(_get_snf_magnitude(a.wave[min_wave_idx:max_wave_idx+1], model, band))
    bandmag = np.array(bandmag)
    # bandflux = np.sum(models, axis=0) / (max_wave - min_wave)
    # bandmag = 2.5*np.log10(bandflux)
    
    return bandmag

In [227]:
colors = {'u': 'C4', 'b': 'C0', 'v': 'C2', 'r': 'C3'}

def plot_lc(idx):
    loc = a.trans[idx]

    target = a.targets[idx]
    scale = 1 / a.model_scales[idx]
    
    p = []
    m = []
    e = []
    c = []
    for raw_spectrum in target.spectra:
        spectrum = raw_spectrum.bin_by_velocity(1000).apply_scale(scale)
        for band in ['u', 'b', 'v', 'r']:
            p.append(spectrum.phase)
            mag, err = spectrum.get_snf_magnitude(band, calculate_error=True)
            m.append(mag)
            e.append(err)
            c.append(colors[band])
            
    print(target)
    print(target.subset)
    print(a.trans[idx])
            
    plt.figure()
    plt.scatter(p, m, c=c)
    plt.errorbar(p, m, e, c=c)
        
    for band in ['u', 'b', 'v', 'r']:
        plt.plot(lc_phases, calc_lc(band, loc), c=colors[band])

    plt.xlim(min_phase, max_phase)
    plt.ylim(-15, -20)
    plt.xlabel('Phase')
    plt.ylabel('Mag')
        
interact(plot_lc, idx=(0, len(a.targets)-1))
    
    
# loc = [0, 0, 0]

# u = calc_lc('u', loc)
# b = calc_lc('b', loc)
# v = calc_lc('v', loc)
# r = calc_lc('r', loc)

# plt.figure()
# plt.plot(lc_phases, u, c='C4', label='u')
# plt.plot(lc_phases, b, c='C0', label='b')
# plt.plot(lc_phases, v, c='C2', label='v')
# plt.plot(lc_phases, r, c='C3', label='r')

# plt.legend()


interactive(children=(IntSlider(value=95, description='idx', max=190), Output()), _dom_classes=('widget-intera…

<function __main__.plot_lc(idx)>

In [213]:
a.spectra[13]

ModifiedSpectrum(target="PTF09dnp", name="PTF09dnp_M001293")

In [214]:
dists = np.sum((a.trans - a.trans[13])**2, axis=1)

In [215]:
np.argsort(dists)

array([ 13,  72, 111, 119,  49, 129,  59,   9, 140,  75,  34,  50,  45,
        92, 102, 138, 113,  38,   3,  37, 106,  27,  87,  17, 154, 131,
        51,  64,  94,  70,  54, 150, 151,  77,  18,  80, 110,  48, 133,
       146, 114, 103,  55,  63,  97,  53, 155, 153,  86,  24, 132,  82,
        21,  57, 144,  79,  28,   5,  66, 125,  41,   4, 130,  56,  58,
       123,  19,  96, 147,  84,   8, 109, 127,   1, 139,  14,   0,  67,
       117,  42, 108, 120,  35, 115,  90,  81, 118, 122,  85,  89,  76,
        62, 101,  36,  10,  93,  46,  12,  98,  99,  30, 141,  65,  39,
        32,  22, 116,   2,  60, 137,  15, 128,   7, 112, 136,  83, 121,
        29,  78,  25,  52, 143,  61,  71,  47,  69,  16,  33, 124,  23,
        26,  11, 126,  40,   6,  68, 149,  95, 104,  73, 152,  43,  31,
       145, 107,  20, 148,  88,  91, 142, 134, 100, 135,  44, 105,  74])

In [248]:
vary_idx = 0
band = 'v'

# center_loc = [-1.5, -2.1, 0.]
center_loc = [0, 1, -1.]

center_loc = np.array(center_loc, dtype=float)
deltas = all_trans - center_loc
diff = np.delete(deltas, vary_idx, axis=1)
dist = np.sqrt(np.sum(diff**2, axis=1))
close_cut = dist < 0.5

close_trans = all_trans[close_cut, vary_idx]

if len(close_trans) > 0:
    min_vary = np.clip(np.min(close_trans), -5, -2)
    max_vary = np.clip(np.max(close_trans), 2, 5)
else:
    print("WARNING: NO DATA IN RANGE")
    min_vary = -2
    max_vary = 2
    
diff = max_vary - min_vary
min_vary -= 0.1*diff
max_vary += 0.1*diff
vary_loc = np.linspace(min_vary, max_vary, 10)

print(min_vary, max_vary)

plt.figure()

for plot_loc in vary_loc:
    use_loc = center_loc.copy()
    use_loc[vary_idx] = plot_loc
    mag = calc_lc(band, use_loc)
    
    plt.plot(
        lc_phases, mag,
        c=plt.cm.coolwarm((plot_loc - min_vary) / (max_vary - min_vary)),
        zorder=-1,
    )
    
# Overplot real data
plot_phases = []
plot_mags = []
plot_colors = []
for idx in np.where(close_cut)[0]:
    phase = all_phases[idx]
    spectrum = all_spectra[idx]
    val = all_trans[idx, vary_idx]
    
    mag = spectrum.get_snf_magnitude(band)
    
    color = plt.cm.coolwarm((val - min_vary) / (max_vary - min_vary))
    
    plot_phases.append(phase)
    plot_mags.append(mag)
    plot_colors.append(color)
    
plt.scatter(plot_phases, plot_mags, c=plot_colors, s=30, edgecolors='k')

plt.ylim(np.nanpercentile(plot_mags, 90) + 0.5, np.nanpercentile(plot_mags, 10) - 0.5)
plt.xlim(min_phase, max_phase)
# plt.ylim(-4, 0)

# plt.legend()

-2.7 5.7


FigureCanvasNbAgg()

(-15, 60)

In [246]:
# Reproduced spectra
vary_idx = 1
center_loc = [0., 0., 0.]
phase = 0

center_loc = np.array(center_loc, dtype=float)

all_model = []

for wave_idx in range(len(a.wave)):
    loc = center_loc.copy()
    loc[vary_idx] = plot_loc
    model = super_spline(all_spl_c[wave_idx], phase, loc)
    all_model.append(model)
    
plt.figure()
plt.plot(a.wave, all_model)

vary_idx = 1
center_loc = [0., 0., 0.]

center_loc = np.array(center_loc, dtype=float)
deltas = all_trans - center_loc
diff = np.delete(deltas, vary_idx, axis=1)
dist = np.sqrt(np.sum(diff**2, axis=1))
close_cut = dist < 1.0

close_trans = all_trans[close_cut, vary_idx]

min_vary = np.min(close_trans)
max_vary = np.max(close_trans)
diff = max_vary - min_vary
min_vary -= 0.1*diff
max_vary += 0.1*diff


plt.figure()
for plot_loc in np.linspace(min_vary, max_vary, 10):
    plot_t = np.linspace(-15, 60, 1000)
    loc = center_loc.copy()
    loc[vary_idx] = plot_loc
    pred_trans = np.array([loc] * len(plot_t))
    
    model = super_spline(spl_c, plot_t, pred_trans)
    
    plt.plot(
        plot_t, model,
        c=plt.cm.coolwarm((plot_loc - min_vary) / (max_vary - min_vary)),
    )

plt.scatter(all_phases[close_cut], all_fluxes[close_cut, wave_idx], c=all_trans[close_cut, vary_idx],
            vmin=min_vary, vmax=max_vary, cmap=plt.cm.coolwarm)
plt.xlim(-20, 40)
# plt.ylim(-5, 25)

ValueError: setting an array element with a sequence.

In [159]:
a.spectra[(np.abs(a.trans[:, 0] - 0.73) < 0.1) & (np.abs(a.trans[:, 1] - 1.8) < 0.1)]

array([], dtype=object)

## GP model

In [223]:
import george
from george import kernels

def build_gp(x):
    kernel = x[0]**2 * kernels.Matern32Kernel([x[1]**2, x[2]**2, x[2]**2], ndim=3)
    gp = george.GP(kernel)
    yerr = x[3] * np.ones(all_fluxes.shape[0])
    gp.compute(interp_x, yerr)
    
    return gp

def to_min(x):
    gp = build_gp(x)
    
    nll = -gp.log_likelihood(all_fluxes[:, 20])
    # gnll = - gp.grad_log_likelihood(all_fluxes[:, 20])

    print(nll, x)
    return nll#, gnll

res = minimize(to_min, [10, 5, 5, 1])#, jac=True)

gp = build_gp([20, 5, 5, 1])

ValueError: Dimension mismatch

In [395]:
gp = build_gp([-7.3473681, 28.69758488,-2.5888035,  0.65813375])

In [396]:
res

      fun: 3484.2829979161615
 hess_inv: array([[ 2.01096038e+00, -5.12541530e+00,  5.82737148e-01,
        -3.63339994e-03],
       [-5.12541530e+00,  1.40694362e+01, -1.51420966e+00,
         1.44636561e-02],
       [ 5.82737148e-01, -1.51420966e+00,  1.78785616e-01,
        -2.02041828e-03],
       [-3.63339994e-03,  1.44636561e-02, -2.02041828e-03,
         2.69526648e-04]])
      jac: array([ 0.003479  ,  0.00067139, -0.00296021,  0.02093506])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 618
      nit: 23
     njev: 101
   status: 2
  success: False
        x: array([-7.3473681 , 28.69758487, -2.5888035 ,  0.65813375])

In [397]:
pred_phase = 0
wave_idx = 110

min_x = np.min(a.trans[:, 0]) - 0.5
max_x = np.max(a.trans[:, 0]) + 0.5
min_y = np.min(a.trans[:, 1]) - 0.5
max_y = np.max(a.trans[:, 1]) + 0.5

num_points = 50

plot_x, plot_y = np.meshgrid(np.linspace(min_x, max_x, num_points),
                             np.linspace(min_y, max_y, num_points))

flat_plot_x = plot_x.flatten()
flat_plot_y = plot_y.flatten()
flat_phases = pred_phase * np.ones(len(flat_plot_x))

plot_coords = np.array([flat_phases, flat_plot_x, flat_plot_y]).T

pred = gp.predict(all_fluxes[:, wave_idx], plot_coords, return_cov=False)
pred = pred.reshape(plot_x.shape)

cut = (all_phases - pred_phase) < 2.5
scatter_fluxes = all_fluxes[cut, wave_idx]

vmin = np.percentile(scatter_fluxes, 5)
vmax = np.percentile(scatter_fluxes, 95)

plt.figure()
plt.imshow(pred[::-1], extent=(min_x, max_x, min_y, max_y), vmin=vmin, vmax=vmax)

plt.scatter(all_trans[cut, 0], all_trans[cut, 1], c=scatter_fluxes,
            edgecolors='k', vmin=vmin, vmax=vmax)


plt.colorbar()

/home/kboone/packages/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [297]:
interp_x.shape

(2474, 3)

In [298]:
all_fluxes.shape

(2474, 144)

In [301]:
pred_x.shape

(1, 3)

In [302]:
plot_coords.shape

(3, 2500)

In [300]:
for transval in np.linspace(-3, 3, 10):
    pred_phases = np.arange(0, 1)
    loc = [0, transval]
    pred_trans = np.array([loc] * len(pred_phases))
    pred_x = np.hstack([pred_phases[:, None], pred_trans])

    all_vals = []
    for i in range(all_fluxes.shape[1]):
        vals, cov = gp.predict(all_fluxes[:, i], pred_x)
        all_vals.append(vals)

    all_vals = np.array(all_vals)
    plt.plot(all_vals, label=loc)
    plt.legend()

KeyboardInterrupt: 

In [290]:
plt.figure()

/home/kboone/packages/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

<Figure size 640x480 with 0 Axes>

In [204]:
plt.figure()
all_vals = np.array(all_vals)
plt.plot(all_vals)
# plt.plot(pred_phases, all_vals)

FigureCanvasNbAgg()

# Interpolation plots

## Examples of interpolations

In [79]:
def plot_same_night(idx, save=False):
    night_flux = a.flux[a.target_map == idx]
    phases = a.salt_phases[a.target_map == idx]
    model = a.interpolation_result['maximum_flux'][idx]
    model_err = a.interpolation_result['maximum_fluxerr'][idx]
    plt.figure()
    for flux, phase in zip(night_flux, phases):
        plt.plot(a.wave, flux, label='%.2f days' % phase)
    plt.plot(a.wave, model, c='k', ls='--', label='Model')
    plt.fill_between(a.wave, model - model_err, model + model_err, facecolor='k', alpha=0.3)
    plt.legend()
    plt.title(a.targets[idx])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Flux')
    
    if save:
        plt.savefig('./figures/interpolation_model_%s.pdf' % a.targets[idx])
    
    # plt.figure()
    # shift_frac = (a.interpolation_result['shift_fluxerr'] / a.interpolation_result['shift_flux'])[a.target_map == idx]
    # plt.plot(a.wave, shift_frac.T)
    # orig_frac = (a.fluxerr / a.flux)[a.target_map == idx]
    # plt.plot(a.wave, orig_frac.T, ls='--')
    
    
    phase_slope = a.interpolation_result['phase_slope']
    phase_quadratic = a.interpolation_result['phase_quadratic']
    gray_offsets = a.interpolation_result['gray_offsets'][a.target_map == idx]
    model_diffs = a.interpolation_result['model_diffs'][a.target_map == idx]
    
    plt.figure()
    for i, (flux, phase, gray_offset, model_diff) in enumerate(zip(night_flux, phases, gray_offsets, model_diffs)):
        plt.plot(a.wave, -2.5*np.log10(flux / model), label='Data %.2f days' % phase, c='C%d' % i)
    for i, (flux, phase, gray_offset, model_diff) in enumerate(zip(night_flux, phases, gray_offsets, model_diffs)):
        plt.plot(a.wave, model_diff, label='Model %.2f days' % phase, c='C%d' % i, ls='--')
    plt.legend(ncol=2, loc=1)
    plt.title(a.targets[idx])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Difference from maximum light (mag)')
    
    if save:
        plt.savefig('./figures/interpolation_difference_%s.pdf' % a.targets[idx])
    
    plt.figure()
    for i, (flux, phase, gray_offset, model_diff) in enumerate(zip(night_flux, phases, gray_offsets, model_diffs)):
        plt.plot(a.wave, -2.5*np.log10(flux / model) - model_diff, label='Residuals %.2f days' % phase, c='C%d' % i)
    plt.legend()
    plt.title(a.targets[idx])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Interpolation residuals (mag)')
    
    if save:
        plt.savefig('./figures/interpolation_residuals_%s.pdf' % a.targets[idx])
    
from ipywidgets import interact
interact(plot_same_night, idx=(0, len(a.targets)-1))

interactive(children=(IntSlider(value=101, description='idx', max=202), Checkbox(value=False, description='sav…

<function __main__.plot_same_night(idx, save=False)>

In [27]:
plot_targets = ['PTF13ayw', 'SN2004gc']
for plot_target in plot_targets:
    target_names = np.array([i.name for i in a.targets])
    plot_idx = np.where(target_names == plot_target)[0][0]

    plot_same_night(plot_idx, save=True)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


...

FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

## Interpolation model

In [78]:
phase_slope = a.interpolation_result['phase_slope']
phase_quadratic = a.interpolation_result['phase_quadratic']
phase_slope_x1 = a.interpolation_result['phase_slope_x1']
phase_quadratic_x1 = a.interpolation_result['phase_quadratic_x1']

def evaluate_phase_difference(phase, x1=0):
    phase_difference = (
        phase_slope * phase
        + phase_quadratic * phase * phase
        + phase_slope_x1 * x1 * phase
        + phase_quadratic_x1 * x1 * phase * phase
    )
    
    return phase_difference

# Look at change in phase for the same x1
max_phase = a.phase_width
min_phase = -a.phase_width
num_phases = 10
phases = np.linspace(min_phase, max_phase, num_phases)

plt.figure()
norm = plt.Normalize(vmin=min_phase, vmax=max_phase)
cmap = plt.cm.Spectral_r
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array(phases)

for phase in phases:
    plt.plot(a.wave, evaluate_phase_difference(phase), c=cmap(norm(phase)))
    
plt.colorbar(sm, label='Phase (days)')

# plt.xlim(-5.2, 5.2)
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Applied difference (mag)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('./figures/interpolation_phase_difference.pdf')


def plot_x1_difference(phase):
    # Look at change in phase for the same x1
    min_x1 = -3
    max_x1 = 2
    num_x1s = 10
    x1s = np.linspace(min_x1, max_x1, num_x1s)

    plt.figure()
    norm = plt.Normalize(vmin=min_x1, vmax=max_x1)
    cmap = plt.cm.Spectral_r
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array(x1s)

    for x1 in x1s:
        plt.plot(a.wave, evaluate_phase_difference(phase, x1) - evaluate_phase_difference(phase, 0), c=cmap(norm(x1)))

    plt.colorbar(sm, label='SALT2 $x_1$')

    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Applied difference relative to $x_1=0$ (mag)')
    plt.title('Difference in interpolation for at %+d days' % phase)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig('./figures/interpolation_x1_difference_phase_%d.pdf' % phase)
    
for phase in [-5, -3, -1, 1, 3, 5]:
    plot_x1_difference(phase)

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


...

FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

## Interpolation uncertainty

In [29]:
try:
    coefs = a.interpolation_result['phase_dispersion_coefficients']
except KeyError:
    coefs = a.stan_data['phase_dispersion_coefficients']
num_phase_coefficients = len(coefs)

def evaluate_phase_dispersion(phase):
    phase_scale = np.abs((num_phase_coefficients / 2) * (phase / a.phase_width))
    full_bins = int(np.floor(phase_scale))
    remainder = phase_scale - full_bins
    
    phase_coefficients = np.zeros(num_phase_coefficients)
    
    for j in range(full_bins + 1):
        if j == full_bins:
            weight = remainder
        else:
            weight = 1
            
        if weight == 0:
            break
            
        if phase > 0:
            phase_bin = num_phase_coefficients // 2 + j
        else:
            phase_bin = num_phase_coefficients // 2 - 1 - j
            
        phase_coefficients[phase_bin] = weight
        
    fractional_dispersion = phase_coefficients.dot(coefs)
    
    # Convert to magnitudes
    mag_dispersion = frac_to_mag(fractional_dispersion)
    
    return mag_dispersion

phases = np.linspace(-a.phase_width, a.phase_width, 1 + num_phase_coefficients)

eval_coefs = np.array([evaluate_phase_dispersion(phase) for phase in phases])

# Uncertainties for different wavelengths
plt.figure()
num_wave = 10
for i in range(num_wave):
    min_wave = a.wave[0]
    max_wave = a.wave[-1]
    wave_range = max_wave - min_wave
    target_wave = min_wave + wave_range * i / (num_wave - 1)
    idx = np.argmin(np.abs(a.wave - target_wave))
    use_wave = a.wave[idx]
    color = plt.cm.rainbow((use_wave - min_wave) / wave_range)
    plt.plot(phases, eval_coefs[:, idx], label='%d $\AA$' % use_wave, c=color)
    
plt.xlim(-5.2, 5.2)
plt.xlabel('Phase (days)')
plt.ylabel('Interpolation uncertainty (mag)')
plt.legend()
plt.savefig('./figures/interpolation_uncertainty_phase.pdf')

plt.figure()
for i in range(len(phases)):
    plt.plot(a.wave, eval_coefs[i], label='%.2f days' % phases[i])
plt.legend()
plt.xlabel('Wavelength $(\AA$)')
plt.ylabel('Interpolation uncertainty (mag)')
plt.savefig('./figures/interpolation_uncertainty_wavelength.pdf')

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

## Model accuracy

In [76]:
max_flux = a.interpolation_result['maximum_flux']
max_fluxerr = a.interpolation_result['maximum_fluxerr']

max_magerr = frac_to_mag(max_fluxerr / max_flux)

rbtl_dispersion = frac_to_mag(a.rbtl_result['fractional_dispersion'])

def plot_uncertainties(show_rbtl=False):
    plt.figure()
    offset = 29
    
    # Make sure that we include the worst offender.
    max_loc = np.argmax(np.sum(max_magerr**2, axis=1))
    start = max_loc % offset
    
    for idx in range(start, len(a.targets), offset):
        plt.plot(a.wave, max_magerr[idx], label=a.targets[idx].name)
    plt.legend(ncol=2)
    
    plt.xlabel('Wavelength ($\AA$)')
    
    if show_rbtl:
        plt.plot(a.wave, rbtl_dispersion, label='Supernova intrinsic dispersion', c='k', lw=2, ls='--')
        plt.ylabel('Dispersion (magnitude)')
        path = './figures/interpolation_uncertainty_rbtl.pdf'
    else:
        plt.ylabel('Interpolation uncertainty (magnitude)')
        path = './figures/interpolation_uncertainty_norbtl.pdf'
        
    plt.legend(ncol=2)
    plt.tight_layout()
    plt.savefig(path)
        
plot_uncertainties(False)
plot_uncertainties(True)


plt.figure()
for idx in range(len(a.targets)):
    if idx == 0:
        label = 'Individual interpolation uncertainties'
    else:
        label = ''
    plt.plot(a.wave, max_magerr[idx], label=label, alpha=0.02, c='C0')
plt.plot(a.wave, rbtl_dispersion, label='Supernova intrinsic dispersion', lw=2, ls='--', c='k')
plt.plot(a.wave, np.median(max_magerr, axis=0), label='Median interpolation uncertainty', lw=2, ls='--', c='C0')
plt.legend()
plt.ylabel('Dispersion (magnitude)')
plt.xlabel('Wavelength ($\AA$)')
plt.tight_layout()
plt.savefig('./figures/interpolation_uncertainty_median.pdf')

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

# Reading between the lines plots

## Show spectra before and after

In [126]:
plt.figure()
plt.plot(a.wave, a.maximum_flux[a.interp_mask][0], c='C0', alpha=1, lw=0.2, label='Individual spectra')
plt.plot(a.wave, a.maximum_flux[a.interp_mask][1:].T, c='C0', alpha=0.5, lw=0.1)
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Flux (arbitrary units)')
plt.legend()
plt.tight_layout()
plt.savefig('./figures/spectra_at_maximum.pdf')

plt.figure()
plt.plot(a.wave, a.scale_flux[a.interp_mask][0], c='C0', alpha=1, lw=2, label='Individual spectra')
plt.plot(a.wave, a.scale_flux[a.interp_mask][1:].T, c='C0', alpha=0.5, lw=0.1)
plt.plot(a.wave, a.mean_flux, c='k', lw=2, ls='--', label='Mean spectrum')
plt.legend()
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Flux (arbitrary units)')
plt.tight_layout()
plt.savefig('./figures/scale_spectra.pdf')

plt.figure()
fractional_dispersion = a.rbtl_result['fractional_dispersion']
plt.plot(a.wave, a.mean_flux, c='k', lw=2, ls='--', label='Mean spectrum')
plt.fill_between(a.wave, a.mean_flux * (1 - fractional_dispersion), a.mean_flux * (1 + fractional_dispersion), label='Supernova intrinsic dispersion', alpha=0.5)
plt.legend()
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Flux (arbitrary units)')
plt.tight_layout()
plt.savefig('./figures/scale_spectra.pdf')

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

## Magnitudes

In [180]:
plt.figure()

plt.scatter(a.redshifts[a.interp_mask], a.mags[a.interp_mask], s=15, c='C3', label='Full sample')
plt.scatter(a.redshifts[a.good_mag_mask], a.mags[a.good_mag_mask], s=15, c='C0', label='Cut sample')
plt.xlabel('Redshift')
plt.ylabel('RBTL measured magnitude')
plt.legend()
plt.tight_layout()
plt.savefig('./figures/rbtl_magnitude.pdf')

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [193]:
print("RBTL mag std:  %.3f mag" % np.std(a.mags[a.good_mag_mask]))
print("RBTL mag NMAD: %.3f mag" % math.nmad(a.mags[a.good_mag_mask]))

RBTL mag std:  0.122 mag
RBTL mag NMAD: 0.112 mag


# SALT2 comparison

In [191]:
plt.figure()
plt.scatter(a.salt_color, a.colors, s=5)
plt.xlabel('SALT2 Color ($c$)')
plt.ylabel('RBTL Color ($A_V$)')
plt.tight_layout()

plt.savefig('./figures/salt2_color_comparison.pdf')

/home/kboone/packages/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()